# Tutorial 4: The Art of Grouping
## Groupby, Aggregation, and Split-Apply-Combine

---

### The Yeller Phenomenon

*It was called Yeller Quarry because of the yellers. In twos and threes and fives, sometimes sevens, very rarely elevens, these beings moved and hunted, expanding to cover ground and prey, contracting to be itself, consume and clean, find slumber.*

*The archivists Grigsu, Yasho, Boffa, and Mink put down the standard line that yellers were not a variety of creature. Anything—provided it was living—could become a yeller grouping. Common bats or cats of the Capital transported to Yeller Quarry had been observed flitting about their boxes in threes, purring off-on in an alternating yeller-style sequence, one cat then another then another, stop and start.*

*The eeriest thing.*

Just as creatures in the Quarry form mysterious groupings, data analysts group data to find patterns. This tutorial teaches you the most powerful technique in pandas: **groupby**.

---

## What You Will Learn

In this tutorial, you will learn to:

1. Use `groupby()` to split data into groups
2. Apply aggregation functions to groups
3. Use multiple aggregations with `.agg()`
4. Group by multiple columns
5. Transform and filter groups

By the end, you will be able to answer questions like:
- What's the average catch value by creature type?
- Which crew has the highest success rate?
- How does revenue vary by sector and month?

---

In [ ]:
import pandas as pd

# Load all our datasets
catches = pd.read_csv('data/catches.csv')
creatures = pd.read_csv('data/creatures.csv')
crews = pd.read_csv('data/crews.csv')
traps = pd.read_csv('data/traps.csv')

# Add total_value column to catches
catches['total_value'] = catches['quantity'] * catches['price_per_unit']

print("All data loaded.")
print(f"Catches: {len(catches)} records")
print(f"Creatures: {len(creatures)} species")
print(f"Crews: {len(crews)} crews")
print(f"Traps: {len(traps)} deployments")

---

## Part 1: The Groupby Concept

**Groupby** follows a three-step pattern:

1. **Split**: Divide the data into groups based on some criteria
2. **Apply**: Apply a function to each group independently
3. **Combine**: Combine the results into a new data structure

This is often called "split-apply-combine."

### Basic Example: Count catches by crew

In [ ]:
# How many catches did each crew make?
catches.groupby('crew_id').size()

The Miasto Trappers Guild (CRW003) made the most catches—they're the oldest and largest crew. The Redmane Expedition (CRW001) and Gull's Remnants (CRW002) together made 14 catches from the Western Marsh.

### Using aggregation functions

In [ ]:
# Total revenue by crew
catches.groupby('crew_id')['total_value'].sum()

In [ ]:
# Average catch value by crew
catches.groupby('crew_id')['total_value'].mean()

The Deep Tunnel Syndicate (CRW006) has the highest average catch value—they work the dangerous deep tunnels where the valuable specimens live. The Senator's Private Catch (CRW009) also shows high averages because they're procuring premium specimens for political events.

---

## Part 2: Common Aggregation Functions

| Function | What it does |
|----------|-------------|
| `sum()` | Total of all values |
| `mean()` | Average |
| `median()` | Middle value |
| `min()` | Minimum |
| `max()` | Maximum |
| `count()` | Number of non-null values |
| `size()` | Number of rows (including nulls) |
| `std()` | Standard deviation |
| `first()` | First value in group |
| `last()` | Last value in group |

In [ ]:
# Various aggregations by creature
print("Total specimens caught per creature:")
print(catches.groupby('creature_id')['quantity'].sum().sort_values(ascending=False).head(10))

In [ ]:
print("\nMax price per unit by creature:")
print(catches.groupby('creature_id')['price_per_unit'].max().sort_values(ascending=False).head(10))

Cave bats (CR011) are caught in the highest quantities—they're abundant and easy to trap. But the Maw Beast (CR005) commands the highest price.

---

## Part 3: Multiple Aggregations with `.agg()`

The `.agg()` method lets you apply multiple functions at once.

In [ ]:
# Multiple stats for each crew
crew_stats = catches.groupby('crew_id')['total_value'].agg(['sum', 'mean', 'max', 'count'])
crew_stats.columns = ['total_revenue', 'avg_catch', 'best_catch', 'num_catches']
crew_stats.sort_values('total_revenue', ascending=False)

In [ ]:
# Different aggregations for different columns
catches.groupby('crew_id').agg({
    'quantity': 'sum',        # Total specimens
    'total_value': 'sum',     # Total revenue
    'price_per_unit': 'mean', # Average price
    'catch_id': 'count'       # Number of catches
}).rename(columns={'catch_id': 'num_catches'})

---

## Part 4: Grouping by Multiple Columns

You can group by more than one column to get finer breakdowns.

In [ ]:
# Revenue by crew AND sector
catches.groupby(['crew_id', 'sector'])['total_value'].sum()

In [ ]:
# Make it more readable with unstack()
catches.groupby(['crew_id', 'sector'])['total_value'].sum().unstack(fill_value=0)

This shows which crews work which sectors. The Grimslew Salvagers (CRW008) only work Grimslew Shore. The Deep Tunnel Syndicate (CRW006) only works Central Quarry. Cidade Trappers Guild (CRW003) focuses on Miasto Sector.

The Western Marsh was worked by the Redmane Expedition and then Gull's Remnants after the disaster.

---

## Part 5: Grouping by Creature Category

To analyze catches by creature category (bird, reptile, etc.), we need to merge in information from the creatures table.

In [ ]:
# Join catches with creature info
catches_with_info = catches.merge(
    creatures[['creature_id', 'common_name', 'category', 'danger_rating', 'metal_content_pct']],
    on='creature_id',
    how='left'
)
catches_with_info.head()

In [ ]:
# Now group by category
catches_with_info.groupby('category').agg({
    'quantity': 'sum',
    'total_value': ['sum', 'mean'],
    'catch_id': 'count'
})

In [ ]:
# Revenue by category, sorted
category_revenue = catches_with_info.groupby('category')['total_value'].sum().sort_values(ascending=False)
print("Total revenue by creature category:")
category_revenue

Birds generate the most revenue—partly because of the extremely valuable yeller bird flock, and partly because birds are easier to catch alive. Reptiles come second, driven by the stone spine lizards and coil tube serpents.

---

## Part 6: Trap Success Rates by Type

Let's analyze trap success rates using groupby.

In [ ]:
# Success rate by trap type
trap_success = traps.groupby('trap_type').agg({
    'successful': ['sum', 'count']
})
trap_success.columns = ['successes', 'total']
trap_success['success_rate'] = trap_success['successes'] / trap_success['total'] * 100
trap_success.sort_values('success_rate', ascending=False)

Cages have the highest success rate—they're designed for live capture of specific specimens. Nets also do well. Boxes have lower rates, perhaps because they're less specialized.

---

## Part 7: Time-Based Grouping

We can extract parts of dates to group by time periods.

In [ ]:
# Convert date string to datetime
catches['date'] = pd.to_datetime(catches['date'])

# Extract month
catches['month'] = catches['date'].dt.month

# Revenue by month
monthly_revenue = catches.groupby('month')['total_value'].sum()
print("Revenue by month:")
print(monthly_revenue)

In [ ]:
# Number of catches by month
monthly_catches = catches.groupby('month').size()
print("\nCatches by month:")
print(monthly_catches)

March had the highest revenue—that's when the Maw Beast was caught, and when the yeller bird flock was delivered. February had the most catches by count.

---

## Part 8: The Complete Crew Analysis

Let's build a comprehensive crew performance report.

In [ ]:
# Merge catches with crew info
catches_crews = catches.merge(
    crews[['crew_id', 'crew_name', 'leader', 'specialty', 'status']],
    on='crew_id',
    how='left'
)

# Comprehensive crew stats
crew_report = catches_crews.groupby(['crew_id', 'crew_name', 'leader']).agg({
    'quantity': 'sum',
    'total_value': ['sum', 'mean', 'max'],
    'catch_id': 'count'
}).round(1)

crew_report.columns = ['total_specimens', 'total_revenue', 'avg_catch_value', 'best_catch', 'num_catches']
crew_report = crew_report.reset_index()
crew_report.sort_values('total_revenue', ascending=False)

The Grimslew Salvagers under Captain Yuro generated the most revenue despite having fewer catches—their wharver specimens command extreme prices. The Miasto Trappers Guild has the highest volume but lower average values.

Note the Redmane Expedition led by The Boss—their total revenue is modest, but their story ended before they could reach their potential.

---

## Exercises

### Exercise 1: Destination Analysis

Group catches by destination. Which destination receives the most specimens? The highest total value?

In [ ]:
# Your code here



### Exercise 2: Danger vs Value

Using the merged catches_with_info DataFrame, group by danger_rating and calculate the average price_per_unit. Do more dangerous creatures command higher prices?

In [ ]:
# Your code here



### Exercise 3: Live vs Dead by Crew

Create a pivot showing total_value for each crew, broken down by condition (live/dead).

In [ ]:
# Your code here
# Hint: groupby(['crew_id', 'condition'])['total_value'].sum().unstack()



### Exercise 4: Sector Productivity

Which sector generates the highest average catch value? The highest total volume?

In [ ]:
# Your code here



### Exercise 5: The Yeller Pattern

Yeller groups form in 2s, 3s, 5s, 7s, and rarely 11s. In the catches data, find all catches where the quantity matches these "yeller numbers." Group by creature_id and count how many yeller-quantity catches each creature has.

*Are certain creatures more likely to be caught in yeller quantities?*

In [ ]:
# Your code here
# Hint: Use .isin([2, 3, 5, 7, 11]) to filter for yeller quantities



---

## Summary

In this tutorial, you learned:

| Concept | Code |
|---------|------|
| Basic groupby | `df.groupby('column')['value'].sum()` |
| Multiple aggregations | `df.groupby('col')['val'].agg(['sum', 'mean'])` |
| Named aggregations | `df.groupby('col').agg({'a': 'sum', 'b': 'mean'})` |
| Group by multiple columns | `df.groupby(['col1', 'col2'])` |
| Unstack for pivot-like view | `grouped.unstack()` |
| Count rows per group | `df.groupby('col').size()` |
| Count non-null values | `df.groupby('col')['val'].count()` |

---

## Next Tutorial

In **Tutorial 5: Joining the Ledgers**, you will learn to merge multiple DataFrames together—combining catches with creatures with crews to answer complex questions that span multiple data sources. This is how real archivists work: cross-referencing documents to build a complete picture.

*"She was the only one who made even a pretense of naming a landmark. The expedition had only found the campsite because she had walked them to within sight of it the day before, gesturing at the hill to order them on to it and then turning off toward a tangent destination of her own."*

*The Boss knew how to bring separate things together. Soon, so will you.*

---